# Advanced usage

### python test class


### config files

Provide option to input config files, in the form I want to get, if they don't provide any, you give/use default values. 
.txt format

Describe it in the documentation how it should look like.

In [1]:
from openTSNE import TSNEEmbedding
from openTSNE import initialization
from openTSNE.callbacks import ErrorLogger

#import utils

import numpy as np
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt


## Load data

In [2]:
from sklearn import datasets

iris = datasets.load_iris()
x, y = iris["data"], iris["target"]

In [3]:
print("Data set contains %d samples with %d features" % x.shape)

Data set contains 150 samples with 4 features


## Create train/test split

In [15]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.33, random_state=42)

In [16]:
print("%d training samples" % x_train.shape[0])
print("%d test samples" % x_test.shape[0])

100 training samples
50 test samples


## Create a t-SNE embedding

Like in the *simple_usage* notebook, we will run the standard t-SNE optimization.

This example shows the standard t-SNE optimization. Much can be done in order to better preserve global structure and improve embedding quality. Please refer to the *preserving_global_structure* notebook for some examples.

**1. Compute the affinities between data points**

In [8]:
class KNNIndex_Elements:
    def __init__(self, metric, metric_params=None, n_jobs=1, random_state=None):
        self.index = None
        self.metric = metric
        self.metric_params = metric_params
        self.n_jobs = n_jobs
        self.random_state = random_state

    def build(self, data):
        """Build the index so we can query nearest neighbors."""

    def query_train(self, data, k):
        """Query the index for the points used to build index."""

    def query(self, query, k):
        """Query the index with new points."""

    def check_metric(self, metric):
        """Check that the metric is supported by the KNNIndex instance."""

### import multi_nearest_neighbors.py

### NearPy

In [8]:
import numpy

from nearpy import Engine
from nearpy.hashes import RandomBinaryProjections

# Dimension of our vector space
dimension = 500

# Create a random binary hash with 10 bits
rbp = RandomBinaryProjections('rbp', 10)

# Create engine with pipeline configuration
engine = Engine(dimension, lshashes=[rbp])

# Index 1000000 random vectors (set their data to a unique string)
for index in range(100000):
    v = numpy.random.randn(dimension)
    engine.store_vector(v, 'data_%d' % index)

# Create random query vector
query = numpy.random.randn(dimension)

# Get nearest neighbours
N = engine.neighbours(query)

RuntimeError: module compiled against API version 0xc but this version of numpy is 0xb

ImportError: numpy.core.multiarray failed to import

In [ ]:
import nearpy
from nearpy.filters import NearestFilter
import sklearn.preprocessing

class NearPy(KNNIndex):
    #VALID_METRICS = neighbors.NearPy.valid_metrics
    #METHOD_PARAMS = neighbors.NearPy.params
    
    def build(self, data):        
        #parameters init
        if self.method_param == None:
            self.method_param = {}
            self.method_param['n_bits'] = 0.0 
            self.method_param['hash_counts'] = 0.0
            #self.method_param['filter'] = NearestFilter(10)
        self._filter = NearestFilter(10)
        self._metric = metric
        
        
        hashes = []

        for k in range(self._hash_counts):
            nearpy_rbp = nearpy.hashes.RandomBinaryProjections(
                'rbp_%d' % k, self._n_bits)
            hashes.append(nearpy_rbp)

        if self._metric == 'euclidean':
            dist = nearpy.distances.EuclideanDistance()
            self._nearpy_engine = nearpy.Engine(
                data.shape[1],
                lshashes=hashes,
                distance=dist)
        else:  # Default (angular) = Cosine distance
            self._nearpy_engine = nearpy.Engine(
                data.shape[1],
                lshashes=hashes,
                vector_filters=[self._filter])

        if self._metric == 'angular':
            data = sklearn.preprocessing.normalize(X, axis=1, norm='l2')
        for i, x in enumerate(data):
            self._nearpy_engine.store_vector(x, i)


    def query_train(self, data, k):
        self._filter.N = k
        if self._metric == 'angular':
            v = sklearn.preprocessing.normalize([v], axis=1, norm='l2')[0]
        return [y for x, y, z in self._nearpy_engine.neighbours(v)]


    def query(self, query, k):
        self._filter.N = k
        if self._metric == 'angular':
            v = sklearn.preprocessing.normalize([v], axis=1, norm='l2')[0]
        return [y for x, y, z in self._nearpy_engine.neighbours(v)]

### ONNG:

In [6]:
#from __future__ import absolute_import
import ngtpy
import numpy as np

class ONNG(KNNIndex):
    #VALID_METRICS = neighbors.ONNG.valid_metrics
    
    def build(self, data):
        #metrics = {'euclidean': '2', 'angular': 'C'}
        self._metric = metrics[metric]
        self._object_type
        self._edge_size_for_search = -2
        self._build_time_limit = 4
        self._epsilon = epsilon
        if self.method_param == None:
            self.method_param['edge'] == 0.0
            self.method_param['outdegree'] == 0.0
            self.method_param['indegree'] == 0.0

        dim = len(data[0])
        index_dir = 'indexes'
        if not os.path.exists(index_dir):
            os.makedirs(index_dir)
        index = os.path.join(
            index_dir,
            'ONNG-{}-{}-{}'.format(self._edge_size, self._outdegree,
                                   self._indegree))
        anngIndex = os.path.join(index_dir, 'ANNG-' + str(self._edge_size))
        if (not os.path.exists(index)) and (not os.path.exists(anngIndex)):
            t = time.time()
            args = ['ngt', 'create', '-it', '-p8', '-b500', '-ga', '-of',
                    '-D' + self._metric, '-d' + str(dim),
                    '-E' + str(self._edge_size), '-S0',
                    '-e' + str(self._epsilon), '-P0', '-B30',
                    '-T' + str(self._build_time_limit), anngIndex]
            subprocess.call(args)
            idx = ngtpy.Index(path=anngIndex)
            idx.batch_insert(X, num_threads=24, debug=False)
            idx.save()
            idx.close()
        if not os.path.exists(index):
            t = time.time()
            args = ['ngt', 'reconstruct-graph', '-mS',
                    '-o ' + str(self._outdegree),
                    '-i ' + str(self._indegree), anngIndex, index]
            subprocess.call(args)
        if os.path.exists(index):
            t = time.time()
            self.index = ngtpy.Index(index, read_only=True)
            self.indexName = index
        else:
            print('ONNG: Problem.')
        
            
    def query_train(self, data, k):    
        neighbors, distances = self.index.search(
            v, k, self._epsilon, self._edge_size_for_search,
            with_distance=False)
        return neighbors, distances
        
        
    def query(self, query, k):
        #check in what format results are, get neighbors, distances
        neighbors, distances = self.index.search(
            v, k, self._epsilon, self._edge_size_for_search,
            with_distance=False)
        return neighbors, distances

ModuleNotFoundError: No module named 'ngtpy'

### FLANN:

In [ ]:
class FLANN(KNNIndex):
    #VALID_METRICS = neighbors.Flann.valid_metrics
    
    def build(self, data):
        #parameters init
        if self.method_param == None:
            self.method_param = {}
            self.method_param['target_precision'] = 0.9  # want 90% target precision  
        self._metric = metric
        
        self.index = pyflann.FLANN(
            target_precision=self._target_precision,
            algorithm='autotuned', 
            log_level='info')
        if self._metric == 'angular':
            data = sklearn.preprocessing.normalize(data, axis=1, norm='l2')
        self.index.build_index(data)


###### 
    def query_train(self, data, k):
        if self._metric == 'angular':
            v = sklearn.preprocessing.normalize([v], axis=1, norm='l2')[0]
        if v.dtype != numpy.float32:
            v = v.astype(numpy.float32)
        return self._flann.nn_index(v, n)[0][0]

    def query(self, query, k):
        if self._metric == 'angular':
            v = sklearn.preprocessing.normalize([v], axis=1, norm='l2')[0]
        if v.dtype != numpy.float32:
            v = v.astype(numpy.float32)
        return self._flann.nn_index(v, n)[0][0]

### RPForest:

In [ ]:
import rpforest
import numpy


class RPForest(KNNIndex):
    #VALID_METRICS = neighbors.Annoy.valid_metrics
    
    def build(self, data):
        n_trees = 5 + int(round((data.shape[0]) ** 0.5 / 20))
        #n_iters = max(5, int(round(np.log2(data.shape[0]))))
        leaf_size = ?
        
        self.index = rpforest.RPForest(leaf_size=leaf_size, no_trees=n_trees)
        
        #if data.dtype != numpy.double:
        #    data = numpy.array(data).astype(numpy.double)
        self.index.fit(data)

    def query_train(self, data, k):
        neighbors, distances = self._model.query(data[0], k)
        return neighbors, distances

    def query(self, query, k):
        neighbors, distances = self._model.query(data[0], k)
        return neighbors, distances

### Hnswlib:

In [28]:
%time

import hnswlib
import numpy as np


class HnswLib(KNNIndex):
    #VALID_METRICS = neighbors.Annoy.valid_metrics
    
    def build(self,data):  
        #parameter init
        #self.metric = {'angular': 'cosine', 'euclidean': 'l2'}[metric]
        if self.method_param == None:
            self.method_param = {}
            self.method_param["efConstruction"] = 200
            self.method_param["M"] = 16
            self.method_param["efRecall"] = 10
            #self.name = 'hnswlib (%s)' % (self.method_param)
        
        
        self.index = hnswlib.Index(space=self.metric, dim=len(data[0]))
        self.index.init_index(max_elements=len(data),
                          ef_construction=self.method_param["efConstruction"],
                          M=self.method_param["M"])
        data_labels = np.arange(len(data))
        self.index.add_items(np.asarray(data), data_labels)
        self.index.set_num_threads(self.n_jobs)
        self.index.set_ef(self.method_param["efRecall"])

    def query(self, query, k):
        neighbors, distances = self.index.knn_query(query, k=k)
        return neighbors, distances
    
    def query_train(self, data, k):
        neighbors, distances = self.index.knn_query(data, k=k)
        return neighbors, distances

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 8.82 µs


ModuleNotFoundError: No module named 'hnswlib'

### Annoy:

In [26]:
%time
from openTSNE.nearest_neighbors import KNNIndex

# Annoy:

import annoy

class Annoy(KNNIndex):
    #VALID_METRICS = neighbors.Annoy.valid_metrics
    
    def build(self, data, k):
        num_items, vector_length = data.shape
        self.index = annoy.AnnoyIndex(vector_length, metric=self.metric)
        #for i, x in enumerate(data):
        #    self.index.add_item(i, x.tolist())
        
        n_trees = 5 + int(round((data.shape[0]) ** 0.5 / 20))
        
        for idx in range(num_items):
            self.index.add_item(idx, data[idx])
        self.index.build(n_trees)
        print(self.index)
    
        
    def query_train(self, data, k):
        #add search_k parameter: tradeoff between speed and accuracy?
        neighbors, distances = self.index.get_nns_by_item(item[0].index[0], n=k, search_k=-1 ,include_distances=True)
        return neighbors, distances


    def query(self, query, k):
        #query ????
        neighbors, distances = self.index.get_nns_by_item(item[0].index[0], n=k, search_k=-1 ,include_distances=True)
        return neighbors, distances

 

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 10 µs


### Test:

In [28]:
affinities_train = MultiANNPerplexityBasedNN(
    x,
    perplexity=30,
    method="annoy",
    metric="euclidean",
    n_jobs=8,
    random_state=42,
)

#possible solution
#affinities_train.knn_index = stelmach.build_knn_index()

print("P:")
print(affinities_train.P)
print("")

print("data:")
print(affinities_train.data)
print("")

print("knn_index:")
print(affinities_train.knn_index)

#print(".index")
#print(affinities_train.index)


TypeError: 'NoneType' object is not iterable

### import affinity_multiann.py

In [29]:
import logging
import operator
from functools import reduce

import numpy as np
import scipy.sparse as sp

#from . import nearest_neighbors

log = logging.getLogger(__name__)

from openTSNE.affinity import PerplexityBasedNN, MultiscaleMixture, FixedSigmaNN, joint_probabilities_nn
from openTSNE import nearest_neighbors

class MultiANNPerplexityBasedNN(PerplexityBasedNN):
    """Compute affinities using nearest neighbors.
    """

    #super init !
    def __init__(
        self,
        data,
        perplexity=30,
        method="exact",
        metric="euclidean",
        metric_params=None,
        symmetrize=True,
        n_jobs=1,
        random_state=None,
    ):
        self.n_samples = data.shape[0]
        self.perplexity = self.check_perplexity(perplexity)

        k_neighbors = min(self.n_samples - 1, int(3 * self.perplexity))
        self.knn_index, self.__neighbors, self.__distances = build_knn_index(
            data, method, k_neighbors, metric, metric_params, n_jobs, random_state
        )

        self.P = joint_probabilities_nn(
            self.__neighbors,
            self.__distances,
            [self.perplexity],
            symmetrize=symmetrize,
            n_jobs=n_jobs,
        )

        self.n_jobs = n_jobs



def build_knn_index(
    data, method, k, metric, metric_params=None, n_jobs=1, random_state=None
):
    methods = {
        #cs: options for ann algorithms
        "exact": nearest_neighbors.BallTree,
        "approx": nearest_neighbors.NNDescent,
        "balltree": nearest_neighbors.BallTree,
        "nndescent": nearest_neighbors.NNDescent,
        #"annoy": multi_nearest_neighbors.Annoy,
        "annoy": Annoy,
        #"": nearest_neighbors.,
        #"": nearest_neighbors.,
    }
    if isinstance(method, nearest_neighbors.KNNIndex):
        knn_index = method

    elif method not in methods:
        raise ValueError(
            "Unrecognized nearest neighbor algorithm `%s`. Please choose one "
            "of the supported methods or provide a valid `KNNIndex` instance."
            % method
        )
    else:
        knn_index = methods[method](
            metric=metric,
            metric_params=metric_params,
            n_jobs=n_jobs,
            random_state=random_state,
        )
    neighbors, distances = knn_index.build(data, k)

    return knn_index, neighbors, distances



class MultiANNFixedSigmaNN(FixedSigmaNN):
    """Compute affinities using using nearest neighbors and a fixed bandwidth
    for the Gaussians in the ambient space.

    Using a fixed Gaussian bandwidth can enable us to find smaller clusters of
    data points than we might be able to using the automatically determined
    bandwidths using perplexity. Note however that this requires mostly trial
    and error.
    """

    def __init__(
        self,
        data,
        sigma,
        k=30,
        method="exact",
        metric="euclidean",
        metric_params=None,
        symmetrize=True,
        n_jobs=1,
        random_state=None,
    ):
        self.n_samples = n_samples = data.shape[0]

        if k >= self.n_samples:
            raise ValueError(
                "`k` (%d) cannot be larger than N-1 (%d)." % (k, self.n_samples)
            )

        knn_index, neighbors, distances = build_knn_index(
            data, method, k, metric, metric_params, n_jobs, random_state
        )

        self.knn_index = knn_index

        # Compute asymmetric pairwise input similarities
        conditional_P = np.exp(-distances ** 2 / (2 * sigma ** 2))
        conditional_P /= np.sum(conditional_P, axis=1)[:, np.newaxis]

        P = sp.csr_matrix(
            (conditional_P.ravel(), neighbors.ravel(), range(0, n_samples * k + 1, k)),
            shape=(n_samples, n_samples),
        )

        # Symmetrize the probability matrix
        if symmetrize:
            P = (P + P.T) / 2

        # Convert weights to probabilities
        P /= np.sum(P)

        self.sigma = sigma
        self.k = k
        self.P = P
        self.n_jobs = n_jobs


class MultiANNMultiscaleMixture(MultiscaleMixture):
    """Calculate affinities using a Gaussian mixture kernel.

    Instead of using a single perplexity to compute the affinities between data
    points, we can use a multiscale Gaussian kernel instead. This allows us to
    incorporate long range interactions.
    """

    def __init__(
        self,
        data,
        perplexities,
        method="exact",
        metric="euclidean",
        metric_params=None,
        symmetrize=True,
        n_jobs=1,
        random_state=None,
    ):
        self.n_samples = data.shape[0]

        # We will compute the nearest neighbors to the max value of perplexity,
        # smaller values can just use indexing to truncate unneeded neighbors
        perplexities = self.check_perplexities(perplexities)
        max_perplexity = np.max(perplexities)
        k_neighbors = min(self.n_samples - 1, int(3 * max_perplexity))

        self.knn_index, self.__neighbors, self.__distances = build_knn_index(
            data, method, k_neighbors, metric, metric_params, n_jobs, random_state
        )

        self.P = self._calculate_P(
            self.__neighbors,
            self.__distances,
            perplexities,
            symmetrize=symmetrize,
            n_jobs=n_jobs,
        )

        self.perplexities = perplexities
        self.n_jobs = n_jobs

class MultiANNMultiscale(MultiANNMultiscaleMixture):
    """Calculate affinities using averaged Gaussian perplexities.

    In contrast to :class:`MultiscaleMixture`, which uses a Gaussian mixture
    kernel, here, we first compute single scale Gaussian kernels, convert them
    to probability distributions, then average them out between scales.

    Please see the :ref:`parameter-guide` for more information.
    """

    @staticmethod
    def _calculate_P(
        neighbors,
        distances,
        perplexities,
        symmetrize=True,
        normalization="pair-wise",
        n_reference_samples=None,
        n_jobs=1,
    ):
        # Compute normalized probabilities for each perplexity
        partial_Ps = [
            joint_probabilities_nn(
                neighbors,
                distances,
                [perplexity],
                symmetrize=symmetrize,
                normalization=normalization,
                n_reference_samples=n_reference_samples,
                n_jobs=n_jobs,
            )
            for perplexity in perplexities
        ]
        # Sum them together, then normalize
        P = reduce(operator.add, partial_Ps, 0)

        # Take care to properly normalize the affinity matrix
        if normalization == "pair-wise":
            P /= np.sum(P)
        elif normalization == "point-wise":
            P = sp.diags(np.asarray(1 / P.sum(axis=1)).ravel()) @ P

        return P

In [19]:
affinities_train = MultiANNPerplexityBasedNN(
    x,
    perplexity=30,
    method="annoy",
    metric="euclidean",
    n_jobs=8,
    random_state=42,
)

#possible solution
#affinities_train.knn_index = stelmach.build_knn_index()

print("P:")
print(affinities_train.P)
print("")

print("data:")
print(affinities_train.data)
print("")

print("knn_index:")
print(affinities_train.knn_index)

#print(".index")
#print(affinities_train.index)


TypeError: 'NoneType' object is not iterable

In [9]:
class dog():
    def bark(self):
        print("woof")

#bello = dog()
#bello.bark()

class deg(dog):
    def berk(self):
        print("dog.bark(self)")
        
#bello.berk()
bello = deg()
bello.bark()
bello.berk()

woof
dog.bark(self)


**2. Generate initial coordinates for our embedding**

In [10]:
%time init_train = initialization.pca(x_train, random_state=42)

NameError: name 'initialization' is not defined

**3. Construct the `TSNEEmbedding` object**

In [11]:
affinities_train = MyPerplexityBasedNN(
    x_train,
    perplexity=30,
    method="approx",
    metric="euclidean",
    n_jobs=8,
    random_state=42,
)

embedding_train = TSNEEmbedding(
    init_train,
    affinities_train,
    negative_gradient_method="fft",
    n_jobs=8,
    callbacks=ErrorLogger(),
)

NameError: name 'MyPerplexityBasedNN' is not defined

In [21]:
class Annoy(BaseANN):
    def __init__(self, metric, n_trees):
        self._n_trees = n_trees
        self._search_k = None
        self._metric = metric

    def fit(self, X):
        self._annoy = annoy.AnnoyIndex(X.shape[1], metric=self._metric)
        for i, x in enumerate(X):
            self._annoy.add_item(i, x.tolist())
        self._annoy.build(self._n_trees)

    def set_query_arguments(self, search_k):
        self._search_k = search_k

    def query(self, v, n):
        return self._annoy.get_nns_by_vector(v.tolist(), n, self._search_k)

    def __str__(self):
        return 'Annoy(n_trees=%d, search_k=%d)' % (self._n_trees,
                                                   self._search_k)

NameError: name 'BaseANN' is not defined

**4. Optimize embedding**

1. Early exaggeration phase

In [24]:
%time embedding_train_1 = embedding_train.optimize(n_iter=250, exaggeration=12, momentum=0.5)

NameError: name 'embedding_train' is not defined

In [25]:
utils.plot(embedding_train_1, y_train)

NameError: name 'utils' is not defined

In [26]:
%time embedding_train_2 = embedding_train_1.optimize(n_iter=750, momentum=0.8)

NameError: name 'embedding_train_1' is not defined

In [27]:
utils.plot(embedding_train_2, y_train, colors=utils.MACOSKO_COLORS)

NameError: name 'utils' is not defined

## Transform

In [29]:
%%time
embedding_test = embedding_train_2.prepare_partial(
    x_test,
    initialization="median",
    k=25,
    perplexity=5,
)

NameError: name 'embedding_train_2' is not defined

In [30]:
utils.plot(embedding_test, y_test, colors=utils.MACOSKO_COLORS)

NameError: name 'utils' is not defined

In [31]:
%%time
embedding_test_1 = embedding_test.optimize(
    n_iter=100,
    learning_rate=1,
    exaggeration=2,
    momentum=0,
)

NameError: name 'embedding_test' is not defined

In [32]:
utils.plot(embedding_test_1, y_test, colors=utils.MACOSKO_COLORS)

NameError: name 'utils' is not defined

## Together

We superimpose the transformed points onto the original embedding with larger opacity.

In [34]:
fig, ax = plt.subplots(figsize=(12, 8))
utils.plot(embedding_train_2, y_train, colors=utils.MACOSKO_COLORS, alpha=0.25, ax=ax)
utils.plot(embedding_test_1, y_test, colo rs=utils.MACOSKO_COLORS, alpha=0.75, ax=ax)

NameError: name 'plt' is not defined